# MobileNetV2 Model

In [ ]:
# Import library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image

# Open file
import os
import PIL
from random import seed
# Model CNN (Deep learning network)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense,\
GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Create function plot loss function and accuracy score graph
def plot_graph(model_values):
    ''' 
    Input : Model_values of keras.callbacks.History
    Return : Graph of Loss function and accuracy score between training dataset and vaildation dataset
    '''
    # Subplots
    fig, ax = plt.subplots(1, 2, figsize=(14,5))
    
    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(model_values.history['loss'], label='Training Loss');
    plt.plot(model_values.history['val_loss'], label='Testing Loss');
    plt.legend(fontsize=12, loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss');
    
    # Plot MSE
    plt.subplot(1, 2, 2)
    
    plt.plot(model_values.history['accuracy'], label='Training Accuracy')
    plt.plot(model_values.history['val_accuracy'], label='Validation Accuracy')
    
    plt.legend(fontsize=12, loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy');

## 00- First check GPU

In [ ]:
# https://www.tensorflow.org/guide/keras/sequential_model
# Due to we use Keras Sequential API, 
# We want to check GPU first before training our model for 
# impore efficiency and reduce time. 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

## 01- Open dataset

In [ ]:
# Define the paths
original_dataset_path = r'C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\poultry_data'
train_path = r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\Train"
validation_path = r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\Validate"
test_path = r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\Test"

## 02 Preprocessing image dataset

In [ ]:
# Set parameter of image 
# https://github.com/keras-team/keras/issues/8090#issuecomment-335155737
batch_size = 50 # Set the batch size for epoch cycle
img_height = 128 # Set the height of the picture
img_width = 128 # Set the width of the picture

# Rescale pixel to reduce image size before using in model
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)
data_gen_test = ImageDataGenerator(rescale=1/255.)

In [ ]:
# Create training dataset
train_dataset = data_gen_train.flow_from_directory(train_path,
                                                   class_mode="categorical",
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size)

# Create validation dataset
valid_dataset = data_gen_valid.flow_from_directory(validation_path,
                                                   class_mode="categorical",
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size)

# Create testing dataset
test_dataset = data_gen_test.flow_from_directory(test_path,
                                                 class_mode="categorical",
                                                 target_size=(img_height, img_width),
                                                 batch_size=batch_size)


## 03 MobileNetV2 model training

### Transfer learning

In [ ]:
# https://keras.io/api/applications/mobilenet/#mobilenetv2-function
# import MobileNetV2 model form keras API
# set input size of image of trianing is 128x128 (smallest size of MobileNetV2)
# due to we want to use transfer learning process 
# we must add `include_top=False` because we wan to add our input data 
# we decide default weigh for mode
mobv2_model = tf.keras.applications.MobileNetV2(input_shape=(128,128,3),
                                                include_top=False, # Transfer learning
                                                weights="imagenet")

In [ ]:
# model summary
# Total params: 2,257,984
# Trainable params: 2,223,872
# Non-trainable params: 34,112
mobv2_model.summary()

In [ ]:
# fix weights and bias 
# train specifically custom head
mobv2_model.trainable=False

### Add custom head and output layers

In [ ]:
# Create output layer 
# We have 4 classes in our output we decide using activation="softmax" 
# for multi classification.
# Before output layer we decide use GlobalAveragePooling2D as 
# one type of flatten layer.
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(mobv2_model.output) # flatten
prediction_layer = tf.keras.layers.Dense(units=4, activation="softmax")(average_pooling_layer)

In [ ]:
# Add Input layer and output layer 
model = tf.keras.models.Model(inputs=mobv2_model.input, 
                                    outputs=prediction_layer)

In [ ]:
# Total params: 2,263,108
# Trainable params: 5,124 # add input layers and  output layers
# Non-trainable params: 2,257,984 --> fix layers
model.summary()

In [ ]:
# Compile the model         
model.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
# Save checkpoints during training
checkpoint_path = r'C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\models\mobilenetv2\mobilenetv2_cp\cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 mode="max",
                                                 verbose=1,
                                                 monitor="val_accuracy")

In [ ]:
# Training model
# make sure you truely save checkpoint_path
history = model.fit(train_dataset,
          epochs=25,
          validation_data=valid_dataset,
          callbacks=[cp_callback])

In [ ]:
# plot graph 
plot_graph(history)


# overfitting between training and validation 
# Final accuracy after training 25 epochs is score in training 0.98% 
# and vaildaion 0.90%
# Loss function after training 25 epochs is score in training 0.06% 
# and vaildaion 0.31%
# Goof perfomance than baseline model 

In [ ]:
# save model
model.save(r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\models\mobilenetv2\mobilenetv2.h5")

In [ ]:
# convert the history.history dict to a pandas DataFrame:    
hist_df = pd.DataFrame(history.history) 
# # save history to csv:  
# hist_csv_file = r"C:\Users\HP\Desktop\Data\poultry_new\model\mobilenetV2\history_mobilenetv2_tf.csv"
# with open(hist_csv_file, mode='w') as f:
#     hist_df.to_csv(f)

# Save history to CSV file
hist_csv_file = r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\models\mobilenetv2\history_mobilenetv2_tl.csv"
hist_df.to_csv(hist_csv_file, index=False)  # Set index=False to exclude the index column

hist_df.head()

### Fine Tuning

In [ ]:
img_height = 128 # Set the height of the picture
img_width = 128 # Set the width of the picture

# load model
mobv2_model = tf.keras.applications.MobileNetV2(input_shape=(img_height, img_width,3),
                                              include_top=False, # Transfer learning
                                              weights="imagenet",
                                              )  

# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(mobv2_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in mobv2_model.layers[:fine_tune_at]:
    layer.trainable = False

# Make sure you have frozen the correct layers
for i, layer in enumerate(mobv2_model.layers):
    if i >= 95:
        print(i, layer.name, layer.trainable)

# Add input layers and output layers
average_pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(mobv2_model.output) # flatten
prediction_layer = tf.keras.layers.Dense(units=4, activation="softmax")(average_pooling_layer)
fineture_model = tf.keras.models.Model(inputs=mobv2_model.input, 
                                     outputs=prediction_layer)

# Compile the model         
fineture_model.compile(loss="categorical_crossentropy", 
                       optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
                       metrics=['accuracy'])

In [ ]:
# Save checkpoints during training
# follow value of vaildation scorce 
checkpoint = tf.keras.callbacks.ModelCheckpoint(r'C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\models\mobilenetv2\ft_cp\mobilenetv2_ft.h5', 
                             monitor= 'val_accuracy', 
                             mode= 'max', 
                             save_best_only = True, 
                             verbose= 1)

In [ ]:
# Training model
# make sure you truely save checkpoint_path
history_ft = fineture_model.fit(train_dataset,  
                             epochs=25, 
                             validation_data=valid_dataset, 
                             callbacks=[checkpoint]) # fine tune continue form transfer learning

In [ ]:
# plot graph 
plot_graph(history_ft)

# Slightly Overfitting between training and validation 
# Final accuracy after fine tuning 25 epochs is up score in training 1.00% (at 23 epoch) 
# and vaildaion 0.93% (better than transfer learning)
# Loss function after training 25 epochs is down score in training 0.01%  (at 23 epoch)
# and vaildaion 0.78% (worse than transfer learning)
# Higher perfomance than baseline model  

In [ ]:
# save model after fine tuning
fineture_model.save(r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\models\mobilenetv2\mobilenetv2_ft.h5")

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history_ft.history) 

# # save to csv: 
# hist_csv_file = '../model/mobilenetV2/history_mobilenetv2_ft.csv'
# with open(hist_csv_file, mode='w') as f:
#     hist_df.to_csv(f)

# Save history to CSV file
hist_csv_file = r"C:\Users\HP\Desktop\Data\poultry_disease_detection_third_iteration\models\mobilenetv2\history_mobilenetv2_ft.csv"
hist_df.to_csv(hist_csv_file, index=False)  # Set index=False to exclude the index column

hist_df.head()